<a href="https://colab.research.google.com/github/ramesh329/Amazon-Fashion-Recommendation-Engine/blob/master/LAB_1_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB 1 (RAG): Create a GenAI Chatbot around your Resume

In this notebook, we will look at pairing a powerful LLM with an advanced
Retrieval-Augmented Generation (RAG) System. The primary and most immediate benefit
from doing so is to "ground" large language models and minize the risk of any
hallunications.

By populating the context of our LLM with specific text data identified as relevant
to the prompt, the model is more capable of being able to respond.

Most of this notebook will leverage h2ogpte, which is a commercial product
available from H2O.

Let's begin!

# Import Required Dependencies

In [1]:
pip install h2ogpte==1.4.10

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 607.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1255 sha256=5ef7c5658746bceb5894cf610d9d39bf6bf5301c59b6ae975a7e6d01bd30dcfe
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2

In [2]:
from h2ogpte import H2OGPTE
import os
import time

# Accessing Enterprise h2oGPT

Enterprise h2oGPTe is an advanced system that can intelligently process text,
audio and image (with text) data. It's built using years of experience with
applying AI to Documents, which has now evolved into a system capable of

As a worked out exercise, we will show how someone with a LinkedIn profile
can download their profile as a PDF and upload it into h2ogpte.

The following screenshots show how to do so in the UI, but we will provide
programmatic examples as well.

## Step 1 - Download any document from the internet.


## Step 2 - Create a new collection
![Image of h2oGPTe](https://i.imgur.com/VMHc4zq.png)

## Step 3 - Name it `MyProfile`
![Image of h2oGPTe](https://i.imgur.com/Z07JF2q.png)

## Step 4 - Upload your PDF
![Image of h2oGPTe](https://i.imgur.com/CDiCgJl.png)


At this point, h2ogpte will begin indexing, chunking and storing your text data
into the vector database.

Once complete, you will be able to begin conversing with it and other possible
interactions.

In the future, you can use the following code example to upload your data as well.

# Programmatic Upload

```python
    collection_id = h2ogpte.create_collection(name="MyProfile", description = "My Experiences")

    f = open("Profile.pdf",'rb')
    uf = h2ogpte.upload("Profile.pdf",f)
    h2ogpte.ingest_uploads(collection_id, [uf])
```

# Step 5 - Visit the UI and Generate an API Key for the Python API

Enterprise h2oGPT comes with a Python client that enables you to leverage the
power of an advanced RAG system in virtually any setting. Any application can
now be enhanced with AI.

In [3]:
# Authenticate

## TODO - UPDATE HERE

rag_url = 'https://h2ogpte.genai-training.h2o.ai/'
rag_key = 'sk-02sPDMKXbQ90byJLbv5RKyms1KlwSvZ8k81vaNQ2l6BrHIeP'

h2ogpte = H2OGPTE(address=rag_url, api_key=rag_key)

In [4]:
# List collections
h2ogpte.list_recent_collections(0, 5)

[CollectionInfo(id='c7945ff7-1afa-4815-9af7-20ffee600c21', name='Test', description='Citigroup reported a decrease in net income for Q1 2024 due to higher expenses, higher cost of credit, and lower revenues, compared to Q1 2023.', document_count=1, document_size=371296, updated_at=datetime.datetime(2024, 7, 12, 7, 6, 11, 607826, tzinfo=TzInfo(UTC)), user_count=1, is_public=False, username='citibe07121085', sessions_count=4),
 CollectionInfo(id='8f89a88c-ba07-4c38-8a84-e296f976d0a4', name='MyProfile', description='Test collection', document_count=1, document_size=4431864, updated_at=datetime.datetime(2024, 7, 10, 7, 38, 27, 740998, tzinfo=TzInfo(UTC)), user_count=1, is_public=True, username='genaicitipune081', sessions_count=2),
 CollectionInfo(id='13cec3d2-2d50-4631-b871-8482e2af6d08', name='Citi Q1 Test', description='Citi Q1', document_count=1, document_size=371296, updated_at=datetime.datetime(2024, 7, 10, 7, 22, 24, 32733, tzinfo=TzInfo(UTC)), user_count=1, is_public=True, username

In [10]:
# Establish Collection Name

## TODO - UPDATE HERE - collection Name
COLLECTION_NAME = 'Test'
collections = h2ogpte.list_recent_collections(0, 20)

# Identify collection id
collection_id = [i.id for i in collections if COLLECTION_NAME.lower() in i.name.lower()][0]
collection_id

'c7945ff7-1afa-4815-9af7-20ffee600c21'

# What is now possible?

Behind the scenes, h2ogpte is powered by several LLMs and other models. This
enables a wide array of possible actions and fascinating ways of using
artificial intelligence for your business or application.

Let's see how well it can understand us and how it can help with career
development.

In [11]:
queries = [
            "What is presented in the document?",
            "What does the report conclude?",
            "Summarize the document."
            ]

In [12]:
for i, q in enumerate(queries):
    print(f"Q{i+1}: {q}")
    chat_session_id = h2ogpte.create_chat_session(collection_id) # This is the collection_id we uploaded
    with h2ogpte.connect(chat_session_id) as session:
        reply = session.query(q,timeout=10600)
        print(f"A{i+1}:{reply.content}")

Q1: What is presented in the document?
A1:The document presents financial information related to Citigroup's first quarter 2024 earnings release. It includes details about Citigroup's net income, cost of credit, revenues, expenses, tangible book value per share, common equity tier 1 (CET1) capital ratio, supplementary leverage ratio (SLR), and other financial metrics. The document also provides information about Citigroup's various business segments, such as Investment Banking, Corporate Lending, Treasury and Trade Solutions, Securities Services, and Wealth. Additionally, the document includes forward-looking statements regarding Citigroup's future financial performance and potential risks and uncertainties that could impact its results.
Q2: What does the report conclude?
A2:The report concludes that Citigroup's net income for the first quarter of 2024 was $3.4 billion, a decrease from $4.6 billion in the prior-year period. The decrease was primarily driven by higher expenses, higher c

# Performing prompt engineering

In [8]:
prompt_query = "According to only the information in the document sources provided within the context above, and answering in short and crisp language, "

for i, q in enumerate(queries):
    print(f"Q{i+1}: {q}")
    chat_session_id = h2ogpte.create_chat_session(collection_id) # This is the collection_id we uploaded
    with h2ogpte.connect(chat_session_id) as session:
        reply = session.query(q,prompt_query=prompt_query,pre_prompt_query="As an AI agent", timeout=10600)
        print(f"A{i+1}:{reply.content}")

Q1: What is presented in the document?
A1:The document presents Citigroup's first quarter 2024 financial results, including net income, revenues, cost of credit, and various financial metrics. It also discusses the impact of organizational simplification initiatives and provides information about Citigroup's business segments and operations. Additionally, the document includes forward-looking statements and information about the company's upcoming conference call.
Q2: What does the report conclude?
A2:The report concludes that Citigroup's net income for the first quarter of 2024 was $3.4 billion, a decrease from the net income of $4.6 billion in the first quarter of 2023. The decrease was primarily driven by higher expenses, higher cost of credit, and lower revenues. The report also highlights the impact of organizational simplification, central bank interest rate policies, regulatory capital rules, inflation, and geopolitical conflicts on Citigroup's financial performance.
Q3: Summari

In [9]:
system_prompt = "You are a helpful AI Assistant and your goal is to extract hashtags"
prompt_query = "According to only the information in the document sources provided within the context above, "
queries = [" Return only 3 hashtags about the person this profile belongs to as a JSON with a single field called hashtags. Do not use the persons name as a hashtag"]

for i, q in enumerate(queries):
    print(f"Q{i+1}: {q}")
    chat_session_id = h2ogpte.create_chat_session(collection_id) # This is the collection_id we uploaded
    with h2ogpte.connect(chat_session_id) as session:
        reply = session.query(q,prompt_query=prompt_query,pre_prompt_query="As an AI agent", system_prompt=system_prompt, timeout=10600)
        print(f"A{i+1}:{reply.content}")

Q1:  Return only 3 hashtags about the person this profile belongs to as a JSON with a single field called hashtags. Do not use the persons name as a hashtag
A1:{
"hashtags": [
"#CitigroupCEO",
"#FinanceIndustry",
"#BusinessLeader"
]
}


# 🎉 **CONGRATULATIONS!** You have completed this lab!
---

# 📚 **EXTENDED TASKS**

We've barely scratched the surface of evaluating Large Language Models here.


# H2O Enterprise GPT

Other notable functionality out of the box

- Embed your own question and get the similarity.
- NLP Extraction Using LLMs
- Text Summarization API
- Choose between Lexical and Semantic Search
- Retrieve specific text chunks - e.g. extract the title from the first chunk
- Return the cosine similarity between two chunks
- When searching, a user can include only certain documents (parameter name is topic id)